In [1]:
import os
base_dir = 'C:\Users\Ben\Documents\GitHub\expression_broad_data'
os.chdir(base_dir) 
# from core import expression_plots 
# from core import io_library 
# # from IPython.core.debugger import Tracer
# import numpy as np
import pandas as pd
# import re
# import matplotlib.pyplot as plt 
# import seaborn as sns
# import csv
# import scipy.cluster.hierarchy as sch
# from collections import Counter, defaultdict
# import math
# import plotly.plotly as py
# py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')
# # Learn about API authentication here: https://plot.ly/pandas/getting-started
# # Find your api_key here: https://plot.ly/settings/api

# #from sklearn.cluster import AgglomerativeClustering

In [ ]:
#Writes Metadata file for KL and SC experiments
#Only needs to be done once

kl_PKA_metadata = [['sample_name', 'strain','NMPP1'],
                      ['AS1_minus', 'AS','minus'],
                      ['AS1_plus', 'AS', 'plus'],
                      ['AS2_minus', 'AS','minus'],
                      ['AS2_plus', 'AS', 'plus'],
                      ['WT_minus', 'WT', 'minus'],
                      ['WT_plus', 'WT', 'plus']]

with open(base_dir + "\expression_data\kl_PKA_as_20160824\kl_NMPP1_metadata.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(kl_PKA_metadata)
    
sc_PKA_metadata = [['sample_name', 'strain','NMPP1'],
                      ['AS_plus','AS', 'plus'],
                      ['WT_minus','WT', 'minus'],
                      ['WT_plus','WT', 'plus']]

with open(base_dir + "\expression_data\SCer_NMPP1_RNA_Seq\sc_NMPP1_metadata.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(sc_PKA_metadata)

#This data and the raw data was then converted to rlog data and DEseq data in R using the file
# deseq2_analysis_KL_PKA.nb
#That file contains Quality Control Plots of the data.  


In [5]:
#Import DE Seq Data, [later rlog and raw count data]
#convert kl_genename to standard format.  
kl_nmpp1_DESEQ = pd.read_csv(base_dir + "\expression_data\kl_PKA_as_20160824\DESEQ_KLac_ASmin_ASplus.csv")
rename_dict = {kl_nmpp1_DESEQ.columns[0]:'kl_genename_GFF'}
kl_nmpp1_DESEQ.rename(columns = rename_dict, inplace = True)

#Convert kl_genename to standard format. 
kl_genename = []
for gene in kl_nmpp1_DESEQ['kl_genename_GFF']:
    if gene[0:5]=='KLLA0':
        new_gene = gene.split('_')[0]+gene.split('_')[1]
    else: 
        new_gene = gene
    kl_genename.append(new_gene)
kl_nmpp1_DESEQ['orf_name'] = kl_genename
kl_nmpp1_lfc = kl_nmpp1_DESEQ[['orf_name', 'log2FoldChange','padj']]
kl_nmpp1_lfc.rename(index=str, columns={"log2FoldChange": "KL_PKA(AS)+1NMPP1"}, inplace = True)
kl_nmpp1_lfc.to_pickle(base_dir + "\expression_data\kl_PKA_as_20160824\LFC_KLac_ASmin_ASplus.pkl")

In [ ]:
#Loading Raw Data - using LFC now
# #Import KL Expression data provided by Kieran
# fname_kl_data = os.path.normpath(base_dir + '\expression_data\kl_PKA_as_20160824\geneExpression.txt')
# kl_PKA_data = pd.read_table(fname_kl_data, index_col = [0])

# #Normalize by the total number of reads
# fname_kl_metadata = os.path.normpath(base_dir + '\expression_data\kl_PKA_as_20160824' + os.sep + '20160824_KL_PKA_AS_SYC_SC_Control_metadata.xlsx')
# kl_PKA_metadata = pd.read_excel(fname_kl_metadata)
# kl_PKA_metadata.set_index('SampleName', inplace = 'True')
# kl_PKA_data_summary = pd.DataFrame(kl_PKA_data.sum(axis = 'rows'),columns = ['Total Aligned Reads'])
# kl_PKA_data_summary['Total Reads'] = kl_PKA_metadata['Total Reads']
# kl_PKA_data_summary['Percent Aligned'] = kl_PKA_data_summary['Total Aligned Reads'] / kl_PKA_metadata['Total Reads']

# kl_PKA_data_norm = pd.DataFrame(index = kl_PKA_data.index)
# #normalize by total aligned reads, adding in a single pseudocount to avoid dividing by zero
# for sample in kl_PKA_data.columns: 
#     kl_PKA_data_norm[sample] = (kl_PKA_data[sample]+1)/kl_PKA_data_summary.loc[sample,'Total Aligned Reads']

# #Average repeat columns
# kl_PKA_data_norm['AS_minus']=kl_PKA_data_norm[['AS1_minus','AS2_minus']].mean(axis=1)  
# kl_PKA_data_norm['AS_plus']=kl_PKA_data_norm[['AS1_plus','AS2_plus']].mean(axis=1)
# kl_PKA_data_norm['AS_log_fold_change']=np.log(kl_PKA_data_norm['AS_plus']/kl_PKA_data_norm['AS_minus'])/np.log(2.0)
# plt.hist(kl_PKA_data_norm['AS_log_fold_change'])

# #Convert Index in kl_PKA_data_norm to joined format. 
# kl_PKA_data_norm.index
# kl_PKA_data_norm_reindex = []
# for gene in kl_PKA_data_norm.index:
#     if gene[0:5]=='KLLA0':
#         new_gene = gene.split('_')[0]+gene.split('_')[1]
#     else: 
#         new_gene = gene
#     kl_PKA_data_norm_reindex.append(new_gene)
# kl_PKA_data_norm_reindex_se = pd.Series(kl_PKA_data_norm_reindex)
# kl_PKA_data_norm['standard orf names'] = kl_PKA_data_norm_reindex_se.values
# kl_PKA_data_norm.set_index('standard orf names', inplace = True)

In [ ]:
#All by all correlations for each condition - This is done in R now

# conditions = ['WT_minus','WT_plus','AS1_minus','AS2_minus','AS1_plus','AS2_plus']
# fig, axarr = plt.subplots(len(conditions), len(conditions), sharex = True, sharey = True)
# for jj in range(len(conditions)): 
#     condition_data_J = kl_PKA_data_norm[conditions[jj]]
#     for kk in range(len(conditions)):
#         ax = axarr[jj,kk]
#         condition_data_K = kl_PKA_data_norm[conditions[kk]]
#         ax.scatter(condition_data_K, condition_data_J)
#         ax.set_xlim(0.0, 0.013)
#         ax.set_xticks([0.0,0.006,0.012])
#         ax.set_ylim(0.0, 0.013)
#         ax.set_yticks([0.0,0.006,0.012])
#         if kk == 0:
#             ax.set_ylabel(conditions[jj], rotation = 45)
#         if jj == 0:
#             ax.set_title(conditions[kk])

        

In [ ]:
#This would be good to do with LFCs but I don't think I have enough repeats.  May need to do it this way when I want to filter 
#out genes effected by NMPP1 alone. 

# #Make WT+/WT- Ratio and AS+/WT- ratio.  Do correlation of all three ratios. 
# kl_PKA_data_norm['WT_log_fold_change'] = np.log(kl_PKA_data_norm['WT_plus']/kl_PKA_data_norm['WT_minus'])/np.log(2.0)
# kl_PKA_data_norm['AS_plus_WT_minus_log_fold_change'] = np.log(kl_PKA_data_norm['AS_plus']/kl_PKA_data_norm['WT_minus'])/np.log(2.0)
# kl_PKA_data_norm['log_AS_fold_change/WT_fold_change']= kl_PKA_data_norm['AS_log_fold_change']-kl_PKA_data_norm['WT_log_fold_change']

# fig2, ax2 = plt.subplots()
# ax2.hist(kl_PKA_data_norm['log_AS_fold_change/WT_fold_change'])

# conditions = ['AS_log_fold_change','WT_log_fold_change', 'AS_plus_WT_minus_log_fold_change','log_AS_fold_change/WT_fold_change']
# fig, axarr = plt.subplots(len(conditions), len(conditions), sharex = True, sharey = True)
# for jj in range(len(conditions)): 
#     condition_data_J = kl_PKA_data_norm[conditions[jj]]
#     for kk in range(len(conditions)):
#         ax = axarr[jj,kk]
#         condition_data_K = kl_PKA_data_norm[conditions[kk]]
#         ax.scatter(condition_data_K, condition_data_J)
# #         ax.set_xlim(0.0, 0.013)
# #         ax.set_xticks([0.0,0.006,0.012])
# #         ax.set_ylim(0.0, 0.013)
# #         ax.set_yticks([0.0,0.006,0.012])
#         if kk == 0:
#             ax.set_ylabel(conditions[jj], rotation = 45)
#         if jj == 0:
#             ax.set_title(conditions[kk])